In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')
ss = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
df = train.copy()
wkpath = '/kaggle/working'
print(train.shape)

### EDA for the dataset

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x=df['target'])

In [ ]:
from sklearn.model_selection import StratifiedKFold, RepeatedKFold, cross_val_score
from lightgbm import LGBMClassifier

In [ ]:
df.describe()

In [ ]:
maps = ['Class_'+str(i) for i in range(1,10)]
targetmaps = {}
for i in range(len(maps)):
    targetmaps[maps[i]]=i
df['target'] = df.target.map(targetmaps)
df['target'].value_counts()

### StratifiedKFold with 5 splits

In [ ]:
skf = StratifiedKFold(n_splits=5)

In [ ]:
Y = df.target.values
for f, (t_, v_) in enumerate(skf.split(X=df, y=Y)):
    df.loc[v_, 'kfold'] = f

df.to_csv(os.path.join(wkpath,'train_5folds.csv'), index=False)

In [ ]:
lgc = LGBMClassifier()
X = df.drop(['id', 'target', 'kfold'], axis=1)
Y = df.target
print(X.shape, Y.shape)


In [ ]:
testcopy = test.drop(['id'], axis=1)
print(X.shape, testcopy.shape)

predict_scores = []

for train_idx, test_idx in skf.split(X, Y):
    print("Running ########")
    xtrain, xval = X.iloc[train_idx], X.iloc[test_idx]
    ytrain, yval = Y.iloc[train_idx], Y.iloc[test_idx]
    lgc.fit(xtrain, ytrain, eval_set=[(xval, yval)], early_stopping_rounds=10)
    ypreds = lgc.predict_proba(testcopy)
    predict_scores.append(ypreds)

In [ ]:
ypreds.shape

In [ ]:
id_arr = pd.DataFrame(data=[str(i) for i in range(200000,300000)], columns=['id'])
id_arr.head()

submission1 = pd.concat([id_arr, pd.DataFrame(ypreds, columns=['Class_'+str(i) for i in range(1,10)])], axis=1)
submission1.head()
submission1.to_csv(os.path.join(wkpath,'lgbm1_crossval.csv'), index=False)

### Trying LGBM Version 2

In [ ]:
lgc2 = LGBMClassifier(learning_rate=0.01, num_iterations=200)

In [ ]:
testcopy = test.drop(['id'], axis=1)
print(X.shape, testcopy.shape)

predict_scores2 = []

for train_idx, test_idx in skf.split(X, Y):
    print("Running ########")
    xtrain, xval = X.iloc[train_idx], X.iloc[test_idx]
    ytrain, yval = Y.iloc[train_idx], Y.iloc[test_idx]
    lgc2.fit(xtrain, ytrain, eval_set=[(xval, yval)], early_stopping_rounds=10)
    ypreds = lgc2.predict_proba(testcopy)
    predict_scores2.append(ypreds)
print(predict_scores2.shape)

In [ ]:
len(predict_scores)

In [ ]:
predict_scores[0]

In [ ]:
yfinal = np.mean(predict_scores, axis=0)
yfinal.shape

In [ ]:
submission2 = pd.concat([id_arr, pd.DataFrame(yfinal, columns=['Class_'+str(i) for i in range(1,10)])], axis=1)
submission2.head()
submission2.to_csv(os.path.join(wkpath,'lgbm2_crossval.csv'), index=False)
